In [26]:
import pandas as pd

In [27]:
def merge_overlap(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []
    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            # test for intersection between lower and higher:
            # we know via sorting that lower[0] <= higher[0]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = (lower[0], upper_bound)  # replace by merged interval
            else:
                merged.append(higher)
    return merged

In [28]:

df_ann = pd.read_csv('/Users/juan/Documents/manu/dev/vms/bio/data/wheatmitedemo.gff', index_col=False, sep='\t', header=None, comment='#')
df_ann.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

df_ref = pd.read_csv('/Users/juan/Documents/manu/dev/vms/bio/MITETracker/extras/data/iwgsc.gff3', index_col=False, sep='\t', header=None, comment='#')
df_ref.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']


In [29]:
total = {}
for k, v in df_ref.iterrows():
    total[v.seqname] = v.end


In [30]:

overlapped = {}
for k, v in df_ann.iterrows():
    overlapped.setdefault(v.seqname, []).append((v.start, v.end))


In [31]:

sequences = df_ref.seqname.unique().tolist()
for seqname in sequences:
    overlapped[seqname] = merge_overlap(overlapped[seqname])


KeyError: 'chrUn'

In [32]:

for seqname in sequences:
    current = overlapped[seqname]
    total_length = 0
    for (start, end) in current:
        total_length += abs(end - start)
    current_ref = df_ref[(df_ref.seqname == seqname)].iloc[0]
    ref_len = abs(current_ref.end - current_ref.start)
    perc = total_length * 100 / ref_len
    print('seqname %s len: %s covered: %s percentage: %s ' % (seqname, ref_len, total_length, perc,))


seqname chr1A len: 594102056 covered: 12699 percentage: 0.0021375115389265713 
seqname chr1B len: 689851870 covered: 10545 percentage: 0.0015285890288302038 
seqname chr1D len: 495453186 covered: 7817 percentage: 0.0015777474483734575 
seqname chr2A len: 780798557 covered: 9181 percentage: 0.0011758474599742376 
seqname chr2B len: 801256715 covered: 14201 percentage: 0.0017723408408502387 
seqname chr2D len: 651852609 covered: 13258 percentage: 0.0020338953648339235 
seqname chr3A len: 750843639 covered: 11022 percentage: 0.0014679487748846735 
seqname chr3B len: 830829764 covered: 9428 percentage: 0.0011347691679471415 
seqname chr3D len: 615552423 covered: 10318 percentage: 0.0016762179165364117 
seqname chr4A len: 744588157 covered: 10296 percentage: 0.0013827778353987439 
seqname chr4B len: 673617499 covered: 13441 percentage: 0.001995346026484386 
seqname chr4D len: 509857067 covered: 5480 percentage: 0.001074811031304191 
seqname chr5A len: 709773743 covered: 14118 percentage: 0.

KeyError: 'chrUn'